<a href="https://colab.research.google.com/github/Alice1304/Section4_project/blob/main/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B84-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 캐글에서 데이터 가져오기

- 용량이 크기 때문에 따로 다운로드 하지 않고 캐글에서 바로 데이터를 불러오기로 함

In [1]:
# 나의 캐글 Account에서 API Token 다운 받기 = > JSON 파일 다운로드 후 캐글과 연결해주기

!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fabiolacheon","key":"e07252d650a626416e5bed159c28df46"}'}

In [2]:
#Kaggle API를 사용하기 위해 json 파일을 ~/.kaggle로 이동

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c word2vec-nlp-tutorial

 99% 51.0M/51.7M [00:03<00:00, 17.8MB/s]
100% 51.7M/51.7M [00:03<00:00, 16.5MB/s]


In [4]:
!ls

kaggle.json  sample_data  word2vec-nlp-tutorial.zip


In [5]:
!unzip /content/word2vec-nlp-tutorial.zip

Archive:  /content/word2vec-nlp-tutorial.zip
  inflating: labeledTrainData.tsv.zip  
  inflating: sampleSubmission.csv    
  inflating: testData.tsv.zip        
  inflating: unlabeledTrainData.tsv.zip  


In [6]:
!unzip /content/labeledTrainData.tsv.zip

Archive:  /content/labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    


In [7]:
!unzip /content/testData.tsv.zip

Archive:  /content/testData.tsv.zip
  inflating: testData.tsv            


In [8]:
!unzip /content/unlabeledTrainData.tsv.zip

Archive:  /content/unlabeledTrainData.tsv.zip
  inflating: unlabeledTrainData.tsv  


In [9]:
#데이터 불러오기 

In [10]:
import pandas as pd
import numpy as np
import warnings
import os
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
SEED = 33

In [11]:
train = pd.read_csv('/content/labeledTrainData.tsv', delimiter='\t')
test = pd.read_csv('/content/testData.tsv', delimiter='\t')

In [12]:
unlabeled_train = pd.read_csv('/content/unlabeledTrainData.tsv', delimiter='\t', error_bad_lines=False)

b'Skipping line 43043: expected 2 fields, saw 3\n'


In [13]:
train.shape, test.shape, unlabeled_train.shape

((25000, 3), (25000, 2), (49998, 2))

In [66]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [14]:
 unlabeled_train.head()

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


## 2. 전처리

In [15]:
#html텍스트가 포함되어 있어 삭제
#불용어 제거

from bs4 import BeautifulSoup
import re

In [17]:
sample = train['review'][0]

In [18]:
sample

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [19]:
soup = BeautifulSoup(sample, 'html.parser')

In [20]:
soup.text  #html삭제

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [21]:
clean = re.sub('[^a-zA-Z]', ' ', soup.text)  #알파벳을 제외한 문자 모두 공백처리

In [22]:
clean

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [23]:
clean.lower()

'with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    mi

In [25]:
!pip install nltk
import nltk 
nltk.download("stopwords")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
from nltk.corpus import stopwords

In [28]:
eng_stopwords = stopwords.words('english')

## 2-1 Lemmatizer

In [78]:
nltk.download("wordnet")
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
#단어를 원형으로 변환해주는 라이브러리

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [79]:
lemmatizer = WordNetLemmatizer()

In [80]:
lemmatizer.lemmatize('runs')

'run'

In [81]:
np.array(all_review_clean)[0].split()

['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [84]:
def process_lemma(sentence):
  return = [lemmatizer.lemmatize(word, 'v') for word in sentence]

In [86]:
# 전처리 과정을 하나의 함수로 출력하기
# 불용어 제외하고 출력하기 & 동사의 원형으로 변환 

def preprocessing(sentence):
  soup = BeautifulSoup(sentence, 'html.parser')
  clean = re.sub('[^a-zA-Z]', ' ', soup.text) 
  clean = clean.lower()
  clean = [word for word in clean.split() if word not in eng_stopwords]
  clean = process_lemma(clean)

  return ' '.join(clean)  #띄어쓰기 넣어서 조인하기 


In [87]:
all_review = pd.concat([train['review'], unlabeled_train['review'],test['review']])

In [88]:
all_review_clean = all_review.apply(preprocessing)

In [89]:
all_review_clean.head() 

0    stuff go moment mj start listen music watch od...
1    classic war worlds timothy hines entertain fil...
2    film start manager nicholas bell give welcome ...
3    must assume praise film greatest film opera ev...
4    superbly trashy wondrously unpretentious explo...
Name: review, dtype: object

## 3. Tokenizer

In [104]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer

## 4. train test split 

- 위에서 합쳐준 데이터들을 다시 분리하기

In [94]:
train_sentences = all_review_cv[:len(train)]
test_sentences = all_review_cv[-len(test):]

In [95]:
train_labels = train['sentiment']

In [96]:
train_labels.shape

(25000,)

In [97]:
train_labels

0        1
1        1
2        0
3        0
4        1
        ..
24995    0
24996    0
24997    0
24998    0
24999    1
Name: sentiment, Length: 25000, dtype: int64

## 5. Model

In [98]:
from sklearn.ensemble import RandomForestClassifier

In [99]:
SEED = 0

In [100]:
rfc = RandomForestClassifier(n_estimators=1000, max_depth=8, n_jobs=-1, random_state = SEED)

In [101]:
rfc.fit(train_sentences, train_labels)

RandomForestClassifier(max_depth=8, n_estimators=1000, n_jobs=-1,
                       random_state=0)

In [102]:
prediction = rfc.predict(test_sentences)

In [103]:
prediction[:10]

array([1, 0, 1, 1, 1, 0, 0, 1, 0, 1])